In [1]:
import tensorflow as tf
import numpy as np

In [ ]:
class modified_gru():
    """
        It's a modified version of normal GRU
    """
    def __init__(self,time_step,init_state,input_seq,scalar_values):
        
        self.time_steps = time_steps              # equal to max no concepts
        self.init_state = init_state              # initial secondary input
        self.input_seq = input_seq                # input seq to be evaluated
        self.scalar_values = scalar_values        # gated scalar values 
        self.input_size = input_size              # size of input vector 
        self.output_size = output_size            # size of output required
        
        # parameters of gru
        self.W_z,self.W_r,self.W_h = tf.get_variable("update_w",[output_size,input_size]),tf.get_variable("reset_w",[output_size,input_size]),tf.get_variable("out_w",[input_size,output_size])
        self.U_z,self.U_r,self.U_h = tf.get_variable("update_u",[input_size,output_size]),tf.get_variable("reset_u",[input_size,output_size]),tf.get_variable("out_u",[output_size,output_size])
        self.b_z,self.b_r,self.b_h = tf.get_variable("update_b",[output_size]),tf.get_variable("reset_b",[output_size]),tf.get_variable("out_b",[output_size])
        
    """
    right now its a normal gru !!!
    You need to change the the output !!!
    """
    def gru_unit(self,h_prev,out=[],steps = self.time_steps):
        index = steps - self.timesteps 
        input_vector = self.input_seq[index]
        # operations
        z_t = tf.nn.sigmoid(tf.matmul(self.W_z,input_vector)+tf.matmul(self.U_z,h_prev)+self.b_z)
        r_t = tf.nn.sigmoid(tf.matmul(self.W_r,input_vector)+tf.matmul(self.U_r,h_prev)+self.b_r)
        h_t = tf.mul(z_t,h_prev)+tf.mul(1.-z_t,tf.nn.sigmoid(tf.matmul(self.W_h,input_vector)+tf.matmul(tf.mul(r_t,h_prev))+self.b_h))
        out.append(h_t)
        self.time_steps -= 1
        if self.time_steps == 0:
            return(out)
        else:
            return gru_unit(h_prev = h_t,out = out,steps = steps) 
    
       
        
        
        
        
        
        
    

In [49]:
output_size = 10
input_size = 10
time_step = 1
def mod_GRU(cell,scalars,concepts):
    OUT = []
    for i,j in zip(scalars,concepts):
        rnn_input = tf.unstack(i,axis = 1)
        print(j)
        out,_ = tf.nn.static_rnn(cell,rnn_input,dtype=tf.float32)
        OUT.append(out)
    return OUT
        

In [50]:
tf.reset_default_graph()
concepts = tf.placeholder(dtype=tf.float32,shape=[None,time_step,input_size])
scalars = tf.placeholder(dtype=tf.float32,shape=[None,time_step,input_size])
cell = tf.nn.rnn_cell.GRUCell(num_units = output_size)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    out = mod_GRU(cell,scalars,concepts)
    feed_dict = {concepts : np.random.random(size = (10,10)),
                 scalars : np.random.random(size = (10))}
    
    OUT = sess.run(out,feed_dict)


TypeError: Tensor objects are not iterable when eager execution is not enabled. To iterate over this tensor use tf.map_fn.

In [6]:
tf.reset_default_graph()
x =[]
for i in range(10):
    x.append(tf.get_variable("var_{}".format(i),[1,2,3]))
for i in x:
    print(i)

<tf.Variable 'var_0:0' shape=(1, 2, 3) dtype=float32_ref>
<tf.Variable 'var_1:0' shape=(1, 2, 3) dtype=float32_ref>
<tf.Variable 'var_2:0' shape=(1, 2, 3) dtype=float32_ref>
<tf.Variable 'var_3:0' shape=(1, 2, 3) dtype=float32_ref>
<tf.Variable 'var_4:0' shape=(1, 2, 3) dtype=float32_ref>
<tf.Variable 'var_5:0' shape=(1, 2, 3) dtype=float32_ref>
<tf.Variable 'var_6:0' shape=(1, 2, 3) dtype=float32_ref>
<tf.Variable 'var_7:0' shape=(1, 2, 3) dtype=float32_ref>
<tf.Variable 'var_8:0' shape=(1, 2, 3) dtype=float32_ref>
<tf.Variable 'var_9:0' shape=(1, 2, 3) dtype=float32_ref>


In [29]:
x = [1,2,3,4,5,6,7]#np.random.random(size=(10,2))
for i in range(1,len(x)):
    print(x[i],x[-i])
    x[i],x[-i] = x[-i],x[i]
print(x)
#     print(x[:-i],'cropped {}'.format(x[-i]))

2 7
3 6
4 5
4 5
3 6
2 7
[1, 2, 3, 4, 5, 6, 7]
